# **A Simple Image Classifier with MobileNet.**

*   To see the performance of the model please run all the cells and check the output of the last cell



In [1]:
# cloning the repository here and getting the required data for training
# and testing
!git clone https://github.com/theUnrealSamurai/simple_classifier.git

Cloning into 'simple_classifier'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 214 (delta 0), reused 214 (delta 0), pack-reused 0
Receiving objects: 100% (214/214), 4.83 MiB | 22.00 MiB/s, done.


In [2]:
# Importing the necessary libraries here and setting the random seed to generate
# the same output results everytime. If the random seed is not set,
# the performance of the model may vary everytime you train it.


import random
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    pd.np.random.seed(seed)
    sklearn.utils.check_random_state(seed)

set_random_seed(13)

<ipython-input-2-37b209f44fc4>:16: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  pd.np.random.seed(seed)


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# path to the folder where the training images are saved.
data_folder = '/content/simple_classifier/data/train_data'

# Defining Image size and batch size here.
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1 # Change the size of the validation dataset here.
                         # Since the size of the dataset is small I'm using 0.1
)

# Generating the Training dataset
train_generator = train_datagen.flow_from_directory(
    data_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Generating the Validation dataset
validation_generator = train_datagen.flow_from_directory(
    data_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Getting the pretrained mobile net model to finetune it for our dataset.
base_model = MobileNet(include_top=False, input_shape=(224, 224, 3))

# Modifying the output nodes to fit for only 7 classes that we have.
# i.e [bar, cherry, elephant, clown, z, grape, bell]
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Training and saving the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 9
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

model.save('trained_model.h5')


Found 174 images belonging to 7 classes.
Found 16 images belonging to 7 classes.
17225924/17225924 [==============================] - 0s 0us/step
Epoch 1/9
6/6 [==============================] - 60s 7s/step - loss: 0.5138 - accuracy: 0.8161 - val_loss: 0.7513 - val_accuracy: 0.8750
Epoch 2/9
6/6 [==============================] - 44s 8s/step - loss: 0.0137 - accuracy: 0.9943 - val_loss: 1.1554 - val_accuracy: 0.8750
Epoch 3/9
6/6 [==============================] - 43s 7s/step - loss: 0.0137 - accuracy: 0.9943 - val_loss: 0.8561 - val_accuracy: 0.8750
Epoch 4/9
6/6 [==============================] - 42s 7s/step - loss: 0.0064 - accuracy: 0.9943 - val_loss: 0.3066 - val_accuracy: 0.9375
Epoch 5/9
6/6 [==============================] - 44s 8s/step - loss: 6.6014e-04 - accuracy: 1.0000 - val_loss: 0.4694 - val_accuracy: 0.9375
Epoch 6/9
6/6 [==============================] - 43s 7s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.1475 - val_accuracy: 0.9375
Epoch 7/9
6/6 [=============

In [4]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('classifier.tflite', 'wb') as f:
    f.write(tflite_model)


In [5]:
!cp /content/classifier.tflite /content/drive/MyDrive/junk
!cp /content/trained_model.h5 /content/drive/MyDrive/junk